In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import pandas as pd
import requests
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import MWETokenizer
import import_ipynb
import c1_extract_paragraphe_issue as c1 
import c2_extract_sentence_issue as c2 
import g1_generate_dictionary as g1
import collections
from collections import Counter
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
Issue  34
importing Jupyter notebook from c2_extract_sentence_issue.ipynb
importing Jupyter notebook from g1_generate_dictionary.ipynb
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def clean_tp(sentence):
    """ Clean the sentence by removing special char."""
    s = sentence.replace("\r\n\s\s+"," ")
    s = s.replace("\r\n"," ")
    s = s.replace('\t','')
    s = s.replace("\s\s+"," ")
    s = s.replace("\\."," ")
    s = s.replace("\\r\\n"," ")
    p = re.compile(r'<.*?>')
    return p.sub('', s)

In [ ]:
def extract_s2e_issue_number(issue_number):
    """ Extract all the sentence with at least one party inside. """

    #List sentences
    #paragraphes = c1.extract_paragraphes_from_issue(number)
    p = c1.extract_paragraphes_from_issue(issue_number)
    sentences = c2.extract_from_txt_sentences(p)
    #list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))]
    list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]
    #Create list that wil contain all the sentences with at least two entities
    sentences_s2 = []

    for i, s in enumerate(sentences):
    #Split line into words with tokenizer to detect entity
        tokens = tokenize_sentence(s,False)

        if(len(set(tokens).intersection(set(ENTITIES)) )> 1):
            sentences_s2.append(s)
    c2.write_sentences(issue_number,sentences_s2)
    return sentences_s2

In [3]:
def tokenize_sentence(sentence, country):
    """Split the sentence in a way that the entities are together and will be able to be detected."""
    # Extract list entities
    list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]
    list_entities = [s.replace('class="textstory"','') for s in list_entities]
    
    tokens_entities = [l.split(' ') for l in list_entities]
    if(country):
        tokens_entities.append(['on','behalf','of'])
        tokens_entities.append(['for'])
        tokens_entities.append(['US','$'])
        tokens_entities.append(['speaking','for'])
    else :

        tokens_entities.append(['on','behalf','of','the'])
        tokens_entities.append(['spoke','with'])
        tokens_entities.append(['on','behalf','of'])
        tokens_entities.append(['for','the'])
        tokens_entities.append(['US','$'])
        tokens_entities.append(['speaking','for','the'])
    tokens_entities.append(['supported','by'])
    tokens_entities.append(['opposed','by'])
    tokenizer1 = MWETokenizer(tokens_entities, separator=' ')
    tokenizer2 = MWETokenizer([['G-77','CHINA']], separator='/')
    tokenizer3 = MWETokenizer([['G-77/',' CHINA']], separator=' ')
    if(type(sentence) == list):
        line = sentence[0].replace(",","")
    else: 
        line = sentence.replace(",","")
    line_splited = word_tokenize(line)
    tokens = tokenizer1.tokenize(line_splited) 
    tokens = tokenizer2.tokenize(tokens) 
    tokens = tokenizer3.tokenize(tokens) 
    tokens = [clean_tp(token) for token in tokens]

    return tokens

In [5]:
def find_pos_tagged_s2e(list_s2e):
    """ Use NLTK to pos_tag all the sentences from list_s2e and return a list of all the sentences pos_tagged"""
    pos_tagged = []
    for s in list_s2e:
        s = s.replace('\\t','')
        s = re.sub(r'\([^)]*\)', '', s)
        tokens = tokenize_sentence(s, False)

        pos_tagged.append(nltk.pos_tag(tokens))  
   
    return pos_tagged

In [ ]:
ENTITIES = list(set([s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]))
COALITIONS = []
SUPPORTS_LINKS = ['with','and','for the','on behalf of the','supported by','speaking for the']
OPPOSITION_LINKS= ['opposed by','while','opposed by the']
LIST_TAGS = ['IN', 'CC', 'NN', 'NNP', 'JJ','NNPS','MD','VBP','VB','VBZ','VBD','RB','VBN','PRP']
PARTY_GROUPINGS = sorted(set([s.replace('\n','').upper() for s in list(open('Text/party_grouping_clean.txt'))] + [s.replace('\n','').title() for s in list(open('Text/party_grouping_clean.txt'))] + [s.replace('\n','') for s in list(open('Text/party_grouping_clean.txt'))]))
list_entities = list(set([s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]))
PARTIES = sorted(set(list_entities).difference(set(PARTY_GROUPINGS)))
